In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
import sys
import yaml
import importlib

import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import holoviews as hv
hv.extension('bokeh')


sys.path.insert(0, os.path.abspath(r'C:/Users/mmccann/repos/bonhoeffer/prey_capture/'))
import paths
import processing_parameters
import functions_bondjango as bd
import functions_misc as fmisc
import functions_matching as fm
import functions_data_handling as fdh
import functions_tuning as tuning
import functions_plotting as fp
import functions_kinematic as fk
import functions_loaders as fl
from wirefree_experiment import WirefreeExperiment, DataContainer
from functions_wirefree_trigger_fix import get_trial_duration_stats, drop_partial_or_long_trials

# fig_path = paths.wf_figures_path
fig_path = r"D:\thesis\figures"

In [ ]:
# Kick out ones that are oin the exps_to_drop.yaml file
with open(paths.file_exclusion_path, 'r') as f:
    # Load the contents of the file into a dictionary
    files_to_exclude = yaml.unsafe_load(f)
    exclude_from_this = files_to_exclude.get('all', [])

# Make histograms of running speeds across all experiments

In [ ]:
frame_rate = processing_parameters.wf_frame_rate

all_paths, all_queries = fl.query_search_list()
mice = ['_'.join(os.path.basename(path).split('_')[7:10]) for path in all_paths[0]]
print(all_paths)

# load the data
data_list = []
path_list = []
for path, queries in zip(all_paths, all_queries):
    
        data, _, metadata  = fl.load_preprocessing(path, queries, latents_flag=False)
        data_list.append(data)
        path_list.append(path)

# data_list = [ds for el in data_list for ds in el]

## Load all preprocessing files

In [ ]:
lower_lim = 0.1
bin = 0.2  # seconds

In [ ]:
# freely moving
running_data = []
# free_data = data_list[0] + data_list[1]
for data in data_list[0]:
    binned_run, _, _  = sp.stats.binned_statistic(data['time_vector'], data['mouse_speed'], 'mean', bins=np.arange(0, len(data['time_vector']), bin))
    running_data.append(binned_run * 100)

running_data = np.concatenate(running_data, axis=0)

# create a histogram with log spaced bins
free_zeros = np.sum(running_data < lower_lim)
print(free_zeros)
counts, edges = np.histogram(running_data[running_data >= lower_lim], bins=np.logspace(np.log10(lower_lim),np.log10(100), 50))
free_zero = hv.Bars(('0', free_zeros)).opts(width=100, height=500, xlabel=None, ylabel=None, color=fp.hv_blue_hex)
# counts = np.concatenate(([free_zeros], counts))
# edges = np.concatenate(([0.001, 0.0015], edges))
free_hist = hv.Histogram((edges, counts)).opts(xlabel=None, ylabel=None, logx=True, logy=False).opts(width=500, height=500, color=fp.hv_blue_hex)

save_path = os.path.join(fig_path, 'running_speed', 'running_speed_hist_free_zero.png')
free_zero = fp.save_figure(free_zero, save_path=save_path, fig_width=1, dpi=800, fontsize='paper', target='screen', display_factor=0.2)

save_path = os.path.join(fig_path, 'running_speed', 'running_speed_hist_free.png')
free_hist = fp.save_figure(free_hist, save_path=save_path, fig_width=5, dpi=800, fontsize='paper', target='save', display_factor=0.2)


In [ ]:
# head fixed
wheel_data = []
# fixed_data = data_list[2] + data_list[3]
for data in data_list[0]:
    data['wheel_speed_abs'] = np.abs(data['wheel_speed'])
    binned_wheel, _, _  = sp.stats.binned_statistic(data['time_vector'], data['wheel_speed_abs'], 'mean', bins=np.arange(0, len(data['time_vector']), bin))
    wheel_data.append(binned_wheel)

wheel_data = np.concatenate(wheel_data, axis=0)

# create a histogram with log spaced bins
wheel_zeros = np.sum(wheel_data < lower_lim)
print(wheel_zeros)
counts, edges = np.histogram(wheel_data[wheel_data >= lower_lim], bins=np.logspace(np.log10(lower_lim),np.log10(100), 50))
# counts = np.concatenate(([wheel_zeros], counts))
# edges = np.concatenate(([0.001, 0.0015], edges))
fixed_zero = hv.Bars(('0', wheel_zeros)).opts(width=100, height=500, xlabel=None, ylabel=None, color='blue')
fixed_hist = hv.Histogram((edges, counts)).opts(xlabel=None, ylabel=None, logx=True, logy=False).opts(width=500, height=500, color='blue')

save_path = os.path.join(fig_path, '20_pupil_diam_wheel_speed', 'session2_running_speed_hist_fixed_zero.png')
fixed_zero = fp.save_figure(fixed_zero, save_path=save_path, fig_width=1, dpi=800, fontsize='paper', target='screen', display_factor=0.2)

save_path = os.path.join(fig_path, '20_pupil_diam_wheel_speed', 'session2_running_speed_hist_fixed.png')
fixed_hist = fp.save_figure(fixed_hist, save_path=save_path, fig_width=5, dpi=800, fontsize='paper', target='save', display_factor=0.2)


In [ ]:
# head fixed
wheel_data_dists = []
wheel_data = []
hist_plot_list = []

for data in data_list[0]:
    data['pupil_diameter'] = np.abs(data['pupil_diameter'])
    binned_wheel, _, _  = sp.stats.binned_statistic(data['time_vector'], data['pupil_diameter'], 'mean', bins=np.arange(0, len(data['time_vector']), bin))
    wheel_data.append(binned_wheel)

wheel_data = np.concatenate(wheel_data, axis=0)
wheel_data_dists.append(np.nan_to_num(wheel_data))
print(np.nanmean(wheel_data))

# create a histogram with log spaced bins
wheel_zeros = np.sum(wheel_data < lower_lim)
print(wheel_zeros)
counts, edges = np.histogram(wheel_data[wheel_data >= lower_lim], bins=np.linspace(10, 300, 50))
fixed_zero = hv.Bars(('0', wheel_zeros)).opts(width=100, height=500, xlabel=None, ylabel=None, color='red')
fixed_hist = hv.Histogram((edges, counts)).opts(xlabel=None, ylabel=None, logx=False, logy=False).opts(width=500, height=500, color='red')

# # save_path = os.path.join(fig_path, '20_pupil_diam_wheel_speed', 'session2_running_speed_hist_fixed_zero.png')
# fixed_zero = fp.save_figure(fixed_zero, save_path=save_path, fig_width=1, dpi=800, fontsize='paper', target='screen', display_factor=0.2)

# # save_path = os.path.join(fig_path, '20_pupil_diam_wheel_speed', 'session2_running_speed_hist_fixed.png')
fixed_hist_0 = fp.save_figure(fixed_hist, save_path=save_path, fig_width=5, dpi=800, fontsize='paper', target='screen', display_factor=0.2)
hist_plot_list.append(fixed_hist_0)



In [ ]:
# head fixed
wheel_data = []

for data in data_list[1]:
    data['pupil_diameter'] = np.abs(data['pupil_diameter'])
    binned_wheel, _, _  = sp.stats.binned_statistic(data['time_vector'], data['pupil_diameter'], 'mean', bins=np.arange(0, len(data['time_vector']), bin))
    wheel_data.append(binned_wheel)

wheel_data = np.concatenate(wheel_data, axis=0)
wheel_data_dists.append(np.nan_to_num(wheel_data))
print(np.nanmean(wheel_data))

# create a histogram with log spaced bins
wheel_zeros = np.sum(wheel_data < lower_lim)
print(wheel_zeros)
counts, edges = np.histogram(wheel_data[wheel_data >= lower_lim], bins=np.linspace(10, 300, 50))
fixed_zero = hv.Bars(('0', wheel_zeros)).opts(width=100, height=500, xlabel=None, ylabel=None, color='blue')
fixed_hist = hv.Histogram((edges, counts)).opts(xlabel=None, ylabel=None, logx=False, logy=False).opts(width=500, height=500, color='blue', alpha=0.5)

# # save_path = os.path.join(fig_path, '20_pupil_diam_wheel_speed', 'session2_running_speed_hist_fixed_zero.png')
# fixed_zero = fp.save_figure(fixed_zero, save_path=save_path, fig_width=1, dpi=800, fontsize='paper', target='screen', display_factor=0.2)

# # save_path = os.path.join(fig_path, '20_pupil_diam_wheel_speed', 'session2_running_speed_hist_fixed.png')
fixed_hist_0 = fp.save_figure(fixed_hist, save_path=save_path, fig_width=5, dpi=800, fontsize='paper', target='screen', display_factor=0.2)
hist_plot_list.append(fixed_hist_0)

In [ ]:
hist_overlay = hv.Overlay(hist_plot_list).opts(width=500, height=500, xlabel='', ylabel='')
save_path = os.path.join(fig_path, '20_pupil_diam_wheel_speed', 'overlay_pupil_diameter_hist_fixed.png')
hist_overlay = fp.save_figure(hist_overlay, save_path=save_path, fig_width=5, dpi=800, fontsize='paper', target='both', display_factor=0.2)

In [ ]:
from scipy.stats import kruskal

In [ ]:
kruskal(*wheel_data_dists)

In [ ]:
both_hists = hv.Overlay([free_hist, fixed_hist]).opts(hv.opts.Histogram(alpha=0.7))
save_path = os.path.join(fig_path, 'running_speed','overlay_histograms.png')
both_hists = fp.save_figure(both_hists, save_path=save_path, fig_width=5, dpi=800, fontsize='paper', target='both', display_factor=0.2)


In [ ]:
both_hists_wo_labels = hv.Overlay([free_hist, fixed_hist]).opts(hv.opts.Histogram(alpha=0.7, xaxis=None, yaxis=None))
save_path = os.path.join(fig_path, 'running_speed','overlay_histograms_wo_labels.png')
both_hists_wo_labels = fp.save_figure(both_hists, save_path=save_path, fig_width=5, dpi=800, fontsize='paper', target='both', display_factor=0.2)


# Check one particular experiment to prototype running onset tuning

In [ ]:
importlib.reload(processing_parameters)

# get the search string
search_string = 'mouse:MM_221109_a, slug:01_11_2023,'
parsed_search = fdh.parse_search_string(search_string)

# get the paths from the database

# get the raw experiment
exp_query = bd.query_database('vr_experiment', search_string)
exp_query.sort(key=lambda x: x['rig'])

# Get the preprocessing file
preproc_query = bd.query_database('analyzed_data', search_string + r', analysis_type:preprocessing')
preproc_query = [q for q in preproc_query if parsed_search['mouse'].lower() in q['slug']]
preproc_query.sort(key=lambda x: x['rig'])
preproc_paths = np.sort(np.array([el['analysis_path'] for el in preproc_query if (el['analysis_type'] == 'preprocessing') and
                        (parsed_search['mouse'].lower() in el['slug'])]))

# Get the tuning curve file
tc_query = bd.query_database('analyzed_data', search_string + r', analysis_type:tc_analysis')
tc_query = [q for q in tc_query if parsed_search['mouse'].lower() in q['slug']]
tc_query.sort(key=lambda x: x['rig'])
tc_paths = np.sort(np.array([el['analysis_path'] for el in tc_query if (el['analysis_type'] == 'tc_analysis') and
                    (parsed_search['mouse'].lower() in el['slug'])]))

# Get the cell matching file
cell_match_query = bd.query_database('analyzed_data', search_string + r', analysis_type:cellmatching')
cell_match_query = [q for q in cell_match_query if parsed_search['mouse'].lower() in q['slug']]
cell_match_query.sort(key=lambda x: x['rig'])
cell_matching_path = np.array([el['analysis_path'] for el in cell_match_query if (el['analysis_type'] == 'cellmatching') and
                                (parsed_search['mouse'].lower() in el['slug']) and ('daycellmatch' in el['slug'])])

# Get the calcium file names
calcium_paths = np.array([p.replace('preproc', 'calciumraw') for p in preproc_paths])

# Parse the rigs
rigs = np.array([os.path.basename(file).split('_')[6] for file in calcium_paths])

print(cell_matching_path)
print(calcium_paths)
print(preproc_paths)
print(tc_paths)
print(rigs)

In [ ]:
# Load the matching assignments and find the column that corresponds to each file
assignments =  fm.match_cells(cell_matching_path[0])
new_cols = [col.split('_')[-2] for col in assignments.columns]
assignments.columns = new_cols
match_cols = [i for i, (rig, col) in enumerate(zip(rigs, assignments.columns)) if str(col) in rig]

# Use number of non-NaNs in each row to filter out components that were not registered in enough sessions
assignments_filtered = assignments.dropna().astype(int).to_numpy()
unassigned = np.array(assignments[np.sum(~np.isnan(assignments), axis=1) < 2])
unassigned = [unassigned[~np.isnan(unassigned[:, 0]), 0].astype(int), unassigned[~np.isnan(unassigned[:, 1]), 1].astype(int)]
unassigned = [np.sort(np.unique(unassigned[0])), np.sort(np.unique(unassigned[1]))]

# Specify the path to the curated cell matches file
curated_cell_matches_path = os.path.join(r"C:\Users\mmccann\Desktop\curated_cell_matches_multi_normal_ALL.xlsx")

try:
    # Read all sheets into a list of dataframes
    curated_matches_dict = pd.read_excel(curated_cell_matches_path, sheet_name=None)

    # Concatenate the dataframes into a single dataframe
    curated_matches = pd.concat(curated_matches_dict.values(), ignore_index=True)

    # Get the hand-picked matches for the current experiment
    day_mouse_curated_idxs = curated_matches[(curated_matches['mouse'] == parsed_search['mouse']) & 
                                            (curated_matches['day'] == parsed_search['slug'])]['index'].values
    day_mouse_curated_matches = assignments_filtered[day_mouse_curated_idxs, :]

except Exception as e:
    print(f"Could not find the file {curated_cell_matches_path}. Continuing with CaImAn matches...")
    day_mouse_curated_matches = assignments_filtered

In [ ]:
# Load freely moving data
exp_free = WirefreeExperiment(exp_info=exp_query[0], preproc_info=preproc_query[0], tc_info=tc_query[0])
exp_free._load_preprocessing()
exp_free._load_tc()

exp_free.dff = tuning.calculate_dff(exp_free.raw_fluor.copy(), baseline_type='quantile', quantile=0.25)
exp_free.norm_dff = tuning.normalize_responses(exp_free.dff.copy())
exp_free.norm_dff = drop_partial_or_long_trials(exp_free.norm_dff)
exp_free.norm_spikes = tuning.normalize_responses(exp_free.inferred_spikes.copy())
exp_free.norm_spikes = drop_partial_or_long_trials(exp_free.norm_spikes)

# load head fixed data
exp_fixed = WirefreeExperiment(exp_info=exp_query[1], preproc_info=preproc_query[1], tc_info=tc_query[1])
exp_fixed._load_preprocessing()
exp_fixed._load_tc()

exp_fixed.dff = tuning.calculate_dff(exp_fixed.raw_fluor.copy(), baseline_type='quantile', quantile=0.25)
exp_fixed.norm_dff = tuning.normalize_responses(exp_fixed.dff.copy())
exp_fixed.norm_dff = drop_partial_or_long_trials(exp_fixed.norm_dff)
exp_fixed.norm_spikes = tuning.normalize_responses(exp_fixed.inferred_spikes.copy())
exp_fixed.norm_spikes = drop_partial_or_long_trials(exp_fixed.norm_spikes)

In [ ]:
def running_triggered_averages(activity, starts, pre_trial=0, post_trial=0):

    activity.dropna(inplace=True)

    bouts = np.array([[idx, idx + int(post_trial*processing_parameters.wf_frame_rate)] for idx in starts])
    bouts[:, 0] -= int(pre_trial * processing_parameters.wf_frame_rate)

    if bouts[-1, 1] > activity.index[-1]:
        bouts[-1, 1] = activity.index[-1]
    if bouts[0, 0] < 0:
        bouts[0, 0] = 0

    # Get the shifts from the zero point (important for plotting)
    max_zero_idx_shift = np.max(bouts[:, 1] - bouts[:, 0])

    traces = []
    for i, frame in enumerate(bouts):
        ds_slice = activity.iloc[frame[0]:frame[1], :].copy()
        ds_slice['trial_num'] = i

        zero_idx_shift = np.abs((frame[1] - frame[0]) - max_zero_idx_shift)
        ds_slice['zero_idx_shift'] = zero_idx_shift

        traces.append(ds_slice)


    traces = pd.concat(traces, axis=0).reset_index(drop=True)
    # traces.drop(['trial_num', 'direction', 'direction_wrapped', 'orientation'], axis=1, inplace=True)
    # traces = traces.groupby('trial_num').agg(list)
    return traces, bouts

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

In [ ]:
min_run_duration = 0.5 * processing_parameters.wf_frame_rate
max_run_sep = 1.0 * processing_parameters.wf_frame_rate
speed_plot = hv.Curve(exp_fixed.kinematics[['time_vector', 'wheel_speed_abs']]).opts(color='k') 

time = exp_fixed.kinematics['time_vector'].to_numpy()
smoothed_wheel_speed = smooth(exp_fixed.kinematics['wheel_speed_abs'], 10)
smoothed_speed_splot = hv.Curve((time, smoothed_wheel_speed)).opts(color='r')

fixed_speed_cutoff = np.percentile(smoothed_wheel_speed, 65)
threshold_plot = hv.HLine(fixed_speed_cutoff).opts(color='blue')

is_running = np.argwhere(smoothed_wheel_speed > fixed_speed_cutoff).squeeze()
running_bouts = [bout for bout in fmisc.consecutive(is_running, stepsize=max_run_sep) if len(bout) >= min_run_duration]
runs = [np.vstack((exp_fixed.kinematics['time_vector'].iloc[run].values, -5*np.ones_like(run))).T for run in running_bouts]
running_start_idxs = np.array([a[0] for a in running_bouts])
running_start_times = exp_fixed.kinematics['time_vector'].iloc[running_start_idxs].values
runs_plot = hv.Path(runs).opts(color='r', line_width=2)

fixed_speed_plot = hv.Overlay([speed_plot, smoothed_speed_splot, runs_plot, threshold_plot]).opts(height=280, width=1400, yaxis=None, xlabel=None, ylabel=None)


# save_path = os.path.join(fig_path, '5_running_evoked_responses', f'fixed_speed_annotated.png')
# fixed_speed_plot = fp.save_figure(fixed_speed_plot, save_path=save_path, fig_width=14, dpi=800, fontsize='paper', target='none', display_factor=0.3)

print(exp_fixed.kinematics['wheel_speed_abs'].min(), exp_fixed.kinematics['wheel_speed_abs'].max())
fixed_speed_plot

In [ ]:
from hmmlearn import hmm


In [ ]:
scores = list()
models = list()
for n_components in range(1, 3):
    for idx in range(10):  # ten different random starting states
        # define our hidden Markov model
        model = hmm.GMMHMM(n_components=n_components, random_state=idx,
                               n_iter=100)
        model.fit(exp_fixed.kinematics['wheel_speed_abs'].to_numpy()[:, None])
        models.append(model)
        scores.append(model.score(exp_fixed.kinematics['wheel_speed_abs'].to_numpy()[:, None]))
      #   print(f'Converged: {model.monitor_.converged}\t\t'
      #         f'Score: {scores[-1]}')

# get the best model
model = models[np.argmax(scores)]
print(f'The best model had a score of {max(scores)} and '
      f'{model.n_components} components')

# use the Viterbi algorithm to predict the most likely sequence of states
# given the model
states = model.predict(exp_fixed.kinematics['wheel_speed_abs'].to_numpy()[:, None])

In [ ]:
predict_speed_plot = hv.Curve((time, states)).opts(color='r')

fixed_speed_plot = hv.Overlay([speed_plot, predict_speed_plot, runs_plot, threshold_plot]).opts(height=280, width=1400, yaxis=None, xlabel=None, ylabel=None)


# save_path = os.path.join(fig_path, '5_running_evoked_responses', f'fixed_speed_annotated.png')
# fixed_speed_plot = fp.save_figure(fixed_speed_plot, save_path=save_path, fig_width=14, dpi=800, fontsize='paper', target='none', display_factor=0.3)

# print(exp_fixed.kinematics['wheel_speed_abs'].min(), exp_fixed.kinematics['wheel_speed_abs'].max())
fixed_speed_plot

In [ ]:
fixed_cells = [col for col in exp_fixed.norm_dff.columns if 'cell' in col]
exp_fixed.cells_to_match = np.random.choice(fixed_cells, 10, replace=False)

In [ ]:
def get_running_modulated_cells(activity_df, cells, running_bouts_idxs, ci_interval=0.95):

    # Find cells that are significantly modulated by running in general
    still_bouts_idxs = np.setdiff1d(activity_df.index, running_bouts_idxs)
    cell_running_activity = activity_df.loc[running_bouts_idxs, cells].apply(np.nanmean, axis=0)
    cell_still_activity = activity_df.loc[still_bouts_idxs, cells].apply(np.nanmean, axis=0)
    running_diff = cell_running_activity - cell_still_activity
    running_cis = sp.stats.t.interval(ci_interval, len(running_diff)-1, loc=np.mean(running_diff), scale=sp.stats.sem(running_diff))
    sig_running_modulated = (running_diff < running_cis[0]) | (running_diff > running_cis[1])

    # Find cells that are significantly modulated by running during visual stimulus
    vis_stim_idxs = activity_df[activity_df.trial_num >= 1].index
    vis_running_idxs = np.intersect1d(running_bouts_idxs, vis_stim_idxs)
    vis_still_idxs = np.intersect1d(still_bouts_idxs, vis_stim_idxs)
    vis_cell_running_activity = activity_df.loc[vis_running_idxs, cells].apply(np.nanmean, axis=0)
    vis_cell_still_activity = activity_df.loc[vis_still_idxs, cells].apply(np.nanmean, axis=0)
    vis_running_diff = vis_cell_running_activity - vis_cell_still_activity
    vis_running_cis = sp.stats.t.interval(ci_interval, len(vis_running_diff)-1, loc=np.mean(vis_running_diff), scale=sp.stats.sem(vis_running_diff))
    sig_vis_running_modulated = (vis_running_diff < vis_running_cis[0]) | (vis_running_diff > vis_running_cis[1])

    df = pd.DataFrame({'run_activity': cell_running_activity, 'still_activity': cell_still_activity, 
                       'run_diff': running_diff, 'sig_run_modulated': sig_running_modulated,
                       'vis_run_activity': cell_running_activity, 'vis_still_activity': cell_still_activity,
                       'vis_run_diff': vis_running_diff, 'sig_vis_run_modulated': sig_vis_running_modulated})
    
    return df
    

In [ ]:
pred_state_idxs = np.argwhere(states > 0).squeeze()

fixed_run_mod = get_running_modulated_cells(exp_fixed.norm_spikes, fixed_cells, pred_state_idxs)
run_mod_idx = fixed_run_mod.sig_run_modulated + fixed_run_mod.sig_vis_run_modulated
col = np.where(run_mod_idx == 2, 'r', np.where(run_mod_idx == 1, 'g', 'k'))

In [ ]:
max_range = np.max([fixed_run_mod.run_activity.max(), fixed_run_mod.still_activity.max()])
x = np.linspace(0, max_range, 100)

plt.figure()
plt.scatter(fixed_run_mod.vis_still_activity, fixed_run_mod.vis_run_activity, c=col, alpha=0.8)
plt.plot(np.linspace(0, max_range, 100), np.linspace(0, max_range, 100), 'k--')
plt.plot(np.linspace(0, max_range, 100), x+0.005, 'k--')
plt.plot(np.linspace(0.005, max_range, 100), np.linspace(0.005, max_range, 100)-0.005, 'k--')
plt.xlim(0, max_range)
plt.ylim(0, max_range)
plt.show()

In [ ]:
pre_run = 1
post_run = 4
pre_trial = int(pre_run * processing_parameters.wf_frame_rate)

run_traces, run_bouts = running_triggered_averages(exp_fixed.norm_dff.copy(), running_starts, pre_trial=pre_run, post_trial=post_run)
idxs_shifts = run_traces.groupby(['trial_num']).apply(lambda x: np.unique(x.zero_idx_shift)[0]).reset_index().rename({0: 'zero_idx_shift'}, axis=1)	
run_traces = run_traces[fixed_cells + ['trial_num', 'zero_idx_shift']].groupby('trial_num').agg(list)

plot_list = []
diff_list = []

for i, cell in enumerate(fixed_cells):
    save_path = os.path.join(fig_path, '5_running_evoked_responses', f'fixed_cell_{i}.png')

    resps = fmisc.list_lists_to_array(run_traces[cell].tolist(), alignment='left')
    mean = np.nanmean(resps, axis=0)

    pre_trial_resp = np.nanmean(resps[:, :pre_trial], axis=1)
    post_run_resp = np.nanmean(resps[:, pre_trial:], axis=1)
    diff = post_run_resp - pre_trial_resp
    diff_list.append(np.nanmean(diff, axis=0))
    
    trials_list = [hv.Curve(resps[r, :]).opts(color='gray', alpha=0.25) for r in np.arange(resps.shape[0])]
    mean_list = [hv.Curve((np.arange(0, mean.shape[0]), mean)).opts(color='r', xlabel='', ylabel='', title=cell, fontsize={'xticks': 10, 'yticks': 10,})]
    vlines = [hv.VLine(int(pre_run * processing_parameters.wf_frame_rate)).opts(color='k', alpha=1, line_width=0.5)]
    run_evoked_plot = hv.Overlay(trials_list + mean_list + vlines).opts(width=150, height=150, yaxis=None, xaxis=None, xlabel=None, ylabel=None, title='')
    # run_evoked_plot = fp.save_figure(run_evoked_plot, save_path=save_path, fig_width=3, dpi=800, fontsize='paper', target='none', display_factor=0.2)
    plot_list.append(run_evoked_plot)

running_trig_avg = hv.Layout(plot_list).cols(10)
running_trig_avg

## Freely Moving

In [ ]:
min_run_duration = 0.5 * processing_parameters.wf_frame_rate
max_run_sep = 1.0 * processing_parameters.wf_frame_rate
speed_plot = hv.Curve(exp_free.kinematics[['time_vector', 'mouse_speed']]).opts(color='k') 

time = exp_free.kinematics['time_vector'].to_numpy()
smoothed_wheel_speed = smooth(exp_free.kinematics['mouse_speed'], 10)
smoothed_speed_splot = hv.Curve((time, smoothed_wheel_speed)).opts(color='r')

fixed_speed_cutoff = np.percentile(smoothed_wheel_speed, 65)
threshold_plot = hv.HLine(fixed_speed_cutoff).opts(color='blue')

is_running = np.argwhere(smoothed_wheel_speed > fixed_speed_cutoff).squeeze()
running_bouts = [bout for bout in fmisc.consecutive(is_running, stepsize=max_run_sep) if len(bout) >= min_run_duration]
runs = [np.vstack((exp_free.kinematics['time_vector'].iloc[run].values, -5*np.ones_like(run))).T for run in running_bouts]
running_start_idxs = np.array([a[0] for a in running_bouts])
running_start_times = exp_free.kinematics['time_vector'].iloc[running_start_idxs].values
runs_plot = hv.Path(runs).opts(color='r', line_width=2)

free_speed_plot = hv.Overlay([speed_plot, smoothed_speed_splot, runs_plot, threshold_plot]).opts(height=280, width=1400, yaxis=None, xlabel=None, ylabel=None)


# save_path = os.path.join(fig_path, '5_running_evoked_responses', f'fixed_speed_annotated.png')
# fixed_speed_plot = fp.save_figure(fixed_speed_plot, save_path=save_path, fig_width=14, dpi=800, fontsize='paper', target='none', display_factor=0.3)

print(exp_free.kinematics['mouse_speed'].min(), exp_free.kinematics['mouse_speed'].max())
free_speed_plot

In [ ]:
scores = list()
models = list()
for n_components in range(1, 3):
    for idx in range(10):  # ten different random starting states
        # define our hidden Markov model
        model = hmm.GMMHMM(n_components=n_components, random_state=idx,
                               n_iter=100)
        model.fit(exp_free.kinematics['mouse_speed'].to_numpy()[:, None])
        models.append(model)
        scores.append(model.score(exp_free.kinematics['mouse_speed'].to_numpy()[:, None]))
      #   print(f'Converged: {model.monitor_.converged}\t\t'
      #         f'Score: {scores[-1]}')

# get the best model
model = models[np.argmax(scores)]
print(f'The best model had a score of {max(scores)} and '
      f'{model.n_components} components')

# use the Viterbi algorithm to predict the most likely sequence of states
# given the model
states = model.predict(exp_free.kinematics['mouse_speed'].to_numpy()[:, None])

In [ ]:
predict_speed_plot = hv.Curve((time, states)).opts(color='r')
is_running = np.argwhere(states > 0).squeeze()

running_bouts = [bout for bout in fmisc.consecutive(is_running, stepsize=max_run_sep) if len(bout) >= 1]
runs = [np.vstack((exp_free.kinematics['time_vector'].iloc[run].values, -5*np.ones_like(run))).T for run in running_bouts]
runs_plot = hv.Path(runs).opts(color='r', line_width=2)

free_speed_plot = hv.Overlay([speed_plot, predict_speed_plot, runs_plot]).opts(height=280, width=1400, yaxis=None, xlabel=None, ylabel=None)
free_speed_plot

In [ ]:
free_cells = [col for col in exp_free.norm_dff.columns if 'cell' in col]
exp_free.cells_to_match = np.random.choice(free_cells, 10, replace=False)

In [ ]:
pre_run = 1
post_run = 4
pre_trial = int(pre_run * processing_parameters.wf_frame_rate)

run_traces, run_bouts = running_triggered_averages(exp_free.norm_dff.copy(), running_starts, pre_trial=pre_run, post_trial=post_run)
idxs_shifts = run_traces.groupby(['trial_num']).apply(lambda x: np.unique(x.zero_idx_shift)[0]).reset_index().rename({0: 'zero_idx_shift'}, axis=1)	
run_traces = run_traces[free_cells + ['trial_num', 'zero_idx_shift']].groupby('trial_num').agg(list)

plot_list = []
diff_list = []

for i, cell in enumerate(free_cells):
    save_path = os.path.join(fig_path, '5_running_evoked_responses', f'free_cell_{i}.png')

    resps = fmisc.list_lists_to_array(run_traces[cell].tolist(), alignment='left')
    mean = np.nanmean(resps, axis=0)

    pre_trial_resp = np.nanmean(resps[:, :pre_trial], axis=1)
    post_run_resp = np.nanmean(resps[:, pre_trial:], axis=1)
    diff = post_run_resp - pre_trial_resp
    diff_list.append(np.nanmean(diff, axis=0))
    
    trials_list = [hv.Curve(resps[r, :]).opts(color='gray', alpha=0.25) for r in np.arange(resps.shape[0])]
    mean_list = [hv.Curve((np.arange(0, mean.shape[0]), mean)).opts(color='r', xlabel='', ylabel='', title=cell, fontsize={'xticks': 10, 'yticks': 10,})]
    vlines = [hv.VLine(int(pre_run * processing_parameters.wf_frame_rate)).opts(color='k', alpha=1, line_width=0.5)]
    run_evoked_plot = hv.Overlay(trials_list + mean_list + vlines).opts(width=150, height=150, yaxis=None, xaxis=None, xlabel=None, ylabel=None, title='')
    # run_evoked_plot = fp.save_figure(run_evoked_plot, save_path=save_path, fig_width=3, dpi=800, fontsize='paper', target='none', display_factor=0.2)
    plot_list.append(run_evoked_plot)

running_trig_avg = hv.Layout(plot_list).cols(10)
running_trig_avg